In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_predict, GridSearchCV, cross_val_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR

In [7]:
from sklearn.feature_selection import SelectFromModel
data = pd.read_csv('Songdata1140-1149.csv')
#data.info()
#labels = ['age', 'Sex', 'Grade', 'Handedness', 'Ground_Time', 'Air_Time', 'Air_Ground_Ratio', 'Speed', 'SD_Time_Per_Word', 'Pressure', 'SD_Pressure', 'Out_Grid', 'Wrong_stroke', 'Additional_stroke', 'Missing_stroke', 'Concatenated_stroke', 'Reversed_stroke', 'Wrong_stroke_sequence', 'Recognized_words', 'Tilt', 'Slant','Shear','HorizontalAlignment','VerticalAlignment','SystematicHorizontalAlignment','SystematicVerticalAlignment','Size','SizeSD','OutofGridExtent','spacingSD']
#'C33', 'C8', 'C29', 'C23', 'C45', 'C7', 'C15', 'C19', 'C83', 'C62', 'C82', 'C9', 'C31', 'C6', 'C17', 'C39', 'C20', 'C76', 'C72', 'C34', 'C24', 'C2', 'C61', 'C12', 'C10', 'C59', 'C18', 'C22', 'C27', 'C115', 'C1', 'C38', 'C32', 'C4', 'C42', 'C14', 'C84', 'C13', 'C64', 'C21', 'C87', 'C81', 'C53', 'A12753', 'A13867', 'A12824', 'A12907', 'A12669', 'A11901', 'A13284', 'A12587', 'A11546', 'A12433', 'A11027', 'A11203', 'A11371', 'A11141', 'A13015', 'A11296', 'A13952', 'A13425', 'A12091', 'A11703', 'A11287', 'E36', 'E50', 'E77', 'E118', 'E14', 'E39', 'E74', 'E101', 'E165', 'E54', 'E130', 'E96', 'E137', 'E44', 'E29', 'E33', 'E94', 'E28', 'E26', 'E139'
#labels=['Lindex', 'friend','enemy', 'c_birthyear', 'c_deathyear', 'c_death_age', 'c_fl_earliest_year', 'c_fl_latest_year', 'fl_age', 'choronym', 'Lu', 'entrytype', 'Longitude', 'Latitude', 'c_status_code']
labelsel=['friend','enemy', 'c_deathyear', 'c_death_age','Latitude', 'c_status_code']
dataset_X = data[labelsel]
dataset_X = dataset_X.as_matrix()
dataset_y = data[['Rank']]
dataset_y = dataset_y.as_matrix().ravel()

C:\Users\mengqili\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':
C:\Users\mengqili\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


In [8]:
# Perform Grid-Search 
gsc = GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid={
        'max_depth': range(3,7),
        'n_estimators': (10, 50, 100, 1000),
    },
    cv=5, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = gsc.fit(dataset_X, dataset_y)
best_params = grid_result.best_params_

sel = SelectFromModel(RandomForestRegressor(max_depth=best_params["max_depth"],
                            n_estimators=best_params["n_estimators"],
                            random_state=False, verbose=False))
sel.fit(dataset_X, dataset_y)
print(sel.get_support())
rfr = RandomForestRegressor(max_depth=best_params["max_depth"],
                            n_estimators=best_params["n_estimators"],
                            random_state=False, verbose=False)
# Perform K-Fold CV
scores = cross_val_score(rfr, dataset_X, dataset_y, cv=10, scoring='neg_mean_absolute_error')
predictions = cross_val_predict(rfr, dataset_X,dataset_y, cv=10)
MSE = metrics.mean_squared_error(dataset_y, predictions)
print(scores)
print(MSE)
#    return predictions

[False False  True False False False]
[-1.1773255  -1.30385952 -1.01584473 -1.68998625 -1.74261291 -1.89114381
 -1.90356995 -1.98615548 -1.64020152 -1.6554788 ]
4.039092188140657


In [9]:
rfr.fit(dataset_X, dataset_y)
print(sorted(zip(map(lambda x: round(x, 4), rfr.feature_importances_), labelsel), reverse=True))

[(0.3153, 'c_deathyear'), (0.1628, 'friend'), (0.1514, 'Latitude'), (0.1319, 'c_status_code'), (0.1245, 'enemy'), (0.1141, 'c_death_age')]


In [18]:
#dataset_y = data[['Rank']]
#dataset_y = dataset_y.as_matrix().ravel()
from sklearn import grid_search
parameters = {'kernel': ('linear', 'rbf','poly'), 'C':[1.5, 10],'gamma': [1e-7, 1e-4],'epsilon':[0.1,0.2,0.5,0.3]}
svr = SVR()
clf = grid_search.GridSearchCV(svr, parameters)
clf.fit(X_train, y_train)
best_params = clf.best_params_
svr = SVR(kernel = best_params['kernel'], C = best_params['C'], gamma= best_params['gamma'], epsilon =  best_params['epsilon'])
X_train, X_test, y_train, y_test = train_test_split(dataset_X, dataset_y, test_size = 0.2, random_state = 42)
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
print(metrics.mean_squared_error(y_test, y_pred))

4.053536846785063


In [19]:
best_params

{'C': 1.5, 'epsilon': 0.3, 'gamma': 0.0001, 'kernel': 'rbf'}